In [2]:
%pwd

'x:\\CampusX\\projects\\end-to-end_laptop_project\\research'

In [3]:
import os
os.chdir("../")

In [4]:
%pwd

'x:\\CampusX\\projects\\end-to-end_laptop_project'

In [15]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataGatheringConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path


In [18]:
from laptop_ml.constants import *
from laptop_ml.utils.common import read_yaml , create_directories

class ConfigurationManager:
    def __init__(self,
                 config_file_path=CONFIG_FILE_PATH,
                 params_file_path=PARAMS_FILE_PATH) :
        self.config=read_yaml(config_file_path)
        self.params=read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])

    def get_data_gathering_config(self)->DataGatheringConfig:
        config=self.config.data_gathering
        create_directories([config.root_dir])

        data_gathering_config=DataGatheringConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file
        )

        return data_gathering_config

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time

In [19]:
class DataGathering:
    def __init__(self,config: DataGatheringConfig):
        self.config=config
        self.driver=webdriver.Chrome()

    def load_page(self):
        self.driver.get(self.config.source_url)
        time.sleep(3)
        old_height = self.driver.execute_script('return document.body.scrollHeight')
        while True:

            self.driver.find_element(by=By.XPATH, value='//*[@id="app"]/main/div[1]/div[3]/div[3]').click()
            time.sleep(2)

            new_height = self.driver.execute_script('return document.body.scrollHeight')

            if new_height == old_height:
                break

            old_height = new_height

    def extract_html(self):
        html = self.driver.page_source

        with open(self.config.local_data_file,'w',encoding='utf-8') as f:
            f.write(html)

    def close_driver(self):
        self.driver.quit()


In [14]:
try:
    config=ConfigurationManager()
    data_gathering_config=config.get_data_gathering_config()
    data_gathering=DataGathering(config=data_gathering_config)
    data_gathering.load_page()
    data_gathering.extract_html()
    data_gathering.close_driver()
except Exception as e:
    raise e